[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/flax/blob/main/docs/guides/flax_basics.ipynb)
[![Open On GitHub](https://img.shields.io/badge/Open-on%20GitHub-blue?logo=GitHub)](https://github.com/google/flax/blob/main/docs/guides/flax_basics.ipynb)

# Flax Basics

This notebook will walk you through the following workflow:

*   Instantiating a model from Flax built-in layers or third-party models.
*   Initializing parameters of the model and manually written training.
*   Using optimizers provided by Flax to ease training.
*   Serialization of parameters and other objects.
*   Creating your own models and managing state.

## Setting up our environment

Here we provide the code needed to set up the environment for our notebook.

In [1]:
# # Install the latest JAXlib version.
# !pip install --upgrade -q pip jax jaxlib
# # Install Flax at head:
# !pip install --upgrade -q git+https://github.com/google/flax.git

In [2]:
# Check GPU
!nvidia-smi

Fri Jan 13 00:41:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install flax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.4/197.4 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 KB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 KB 12.4 MB/s eta 0:00:00


In [4]:
import jax, flax
from typing import Any, Callable, Sequence
from jax import lax, random, numpy as jnp
from flax.core import freeze, unfreeze
from flax import linen as nn

In [5]:
jax.__version__, flax.__version__

('0.3.25', '0.6.3')

## Linear regression with Flax

In the previous *JAX for the impatient* notebook, we finished up with a linear regression example. As we know, linear regression can also be written as a single dense neural network layer, which we will show in the following so that we can compare how it's done.

A dense layer is a layer that has a kernel parameter $W\in\mathcal{M}_{m,n}(\mathbb{R})$ where $m$ is the number of features as an output of the model, and $n$ the dimensionality of the input, and a bias parameter $b\in\mathbb{R}^m$. The dense layers returns $Wx+b$ from an input $x\in\mathbb{R}^n$.

This dense layer is already provided by Flax in the `flax.linen` module (here imported as `nn`).

In [6]:
# We create one dense layer instance (taking 'features' parameter as input)
model = nn.Dense(features=5)

In [7]:
model

Dense(
    # attributes
    features = 5
    use_bias = True
    dtype = None
    param_dtype = float32
    precision = None
    kernel_init = init
    bias_init = zeros
)

Layers (and models in general, we'll use that word from now on) are subclasses of the `linen.Module` class.

### Model parameters & initialization

Parameters are not stored with the models themselves. You need to initialize parameters by calling the `init` function, using a PRNGKey and dummy input data.

In [8]:
key1, key2 = random.split(random.PRNGKey(0))
x = random.normal(key1, (10,)) # Dummy input data
params = model.init(key2, x) # Initialization call
jax.tree_util.tree_map(lambda x: x.shape, params) # Checking output shapes

FrozenDict({
    params: {
        bias: (5,),
        kernel: (10, 5),
    },
})

In [9]:
x.shape

(10,)

(10, 5)^T (10,) = (5) + (5) bias (5 neurons)

In [10]:
params

FrozenDict({
    params: {
        kernel: DeviceArray([[ 2.35571593e-01, -1.71652630e-01, -4.45728898e-02,
                      -4.68043625e-01,  4.54595298e-01],
                     [-6.87736511e-01,  3.67835432e-01, -1.79262117e-01,
                       1.29276216e-01, -2.42580175e-01],
                     [ 2.02303097e-01, -2.49465629e-01,  2.74955630e-01,
                       4.73488301e-01, -1.98002532e-01],
                     [ 2.74478376e-01, -1.21369645e-01, -2.25361690e-01,
                      -4.78193611e-01, -9.63979959e-02],
                     [-6.19886220e-02, -1.72743499e-01,  2.96947401e-04,
                      -7.17593431e-01,  2.00894251e-01],
                     [-5.60321212e-01,  3.27208459e-01,  1.06281511e-01,
                       1.28758654e-01,  1.16973273e-01],
                     [ 1.82219014e-01,  1.11444041e-01, -1.62924170e-01,
                       3.24953273e-02, -1.67053357e-01],
                     [ 4.31294084e-01,  2.08004534e-01,

*Note: JAX and Flax, like NumPy, are row-based systems, meaning that vectors are represented as row vectors and not column vectors. This can be seen in the shape of the kernel here.*

The result is what we expect: bias and kernel parameters of the correct size. Under the hood:

*   The dummy input data `x` is used to trigger shape inference: we only declared the number of features we wanted in the output of the model, not the size of the input. Flax finds out by itself the correct size of the kernel.
*   The random PRNG key is used to trigger the initialization functions (those have default values provided by the module here).
* Initialization functions are called to generate the initial set of parameters that the model will use. Those are functions that take as arguments `(PRNG Key, shape, dtype)` and return an Array of shape `shape`.
* The init function returns the initialized set of parameters (you can also get the output of the forward pass on the dummy input with the same syntax by using the `init_with_output` method instead of `init`.

In [11]:
import numpy as np

In [12]:
np.ones((3))

array([1., 1., 1.])

The output shows that the parameters are stored in a `FrozenDict` instance, which helps deal with the functional nature of JAX by preventing any mutation of the underlying dict and making the user aware of it. Read more about it in the [`flax.core.frozen_dict.FrozenDict` API docs](https://flax.readthedocs.io/en/latest/api_reference/flax.core.frozen_dict.html#flax.core.frozen_dict.FrozenDict).

As a consequence, the following doesn't work:

In [13]:
try:
    params['new_key'] = jnp.ones((2,2))
except ValueError as e:
    print("Error: ", e)

Error:  FrozenDict is immutable.


To conduct a forward pass with the model with a given set of parameters (which are never stored with the model), we just use the `apply` method by providing it the parameters to use as well as the input:

In [14]:
model.apply(params, x)

DeviceArray([-1.3721197 ,  0.61131513,  0.6442838 ,  2.2192965 ,
             -1.1271117 ], dtype=float32)

### Gradient descent

If you jumped here directly without going through the JAX part, here is the linear regression formulation we're going to use: from a set of data points $\{(x_i,y_i), i\in \{1,\ldots, k\}, x_i\in\mathbb{R}^n,y_i\in\mathbb{R}^m\}$, we try to find a set of parameters $W\in \mathcal{M}_{m,n}(\mathbb{R}), b\in\mathbb{R}^m$ such that the function $f_{W,b}(x)=Wx+b$ minimizes the mean squared error:

$$\mathcal{L}(W,b)\rightarrow\frac{1}{k}\sum_{i=1}^{k} \frac{1}{2}\|y_i-f_{W,b}(x_i)\|^2_2$$

Here, we see that the tuple $(W,b)$ matches the parameters of the Dense layer. We'll perform gradient descent using those. Let's first generate the fake data we'll use. The data is exactly the same as in the JAX part's linear regression pytree example.

In [15]:
# Set problem dimensions.
n_samples = 20
x_dim = 10
y_dim = 5

# Generate random ground truth W and b.
key = random.PRNGKey(0)
k1, k2 = random.split(key)
W = random.normal(k1, (x_dim, y_dim))
b = random.normal(k2, (y_dim,))
# Store the parameters in a FrozenDict pytree.
true_params = freeze({'params': {'bias': b, 'kernel': W}})

# Generate samples with additional noise.
key_sample, key_noise = random.split(k1)
x_samples = random.normal(key_sample, (n_samples, x_dim))
y_samples = jnp.dot(x_samples, W) + b + 0.1 * random.normal(key_noise,(n_samples, y_dim))
print('x shape:', x_samples.shape, '; y shape:', y_samples.shape)

x shape: (20, 10) ; y shape: (20, 5)


In [16]:
x_samples.shape, W.shape, jnp.dot(x_samples, W).shape

((20, 10), (10, 5), (20, 5))

We copy the same training loop that we used in the JAX pytree linear regression example with `jax.value_and_grad()`, but here we can use `model.apply()` instead of having to define our own feed-forward function (`predict_pytree()` in the [JAX example](https://flax.readthedocs.io/en/latest/guides/jax_for_the_impatient.html#linear-regression-with-pytrees)).

In [17]:
# Same as JAX version but using model.apply().
@jax.jit
def mse(params, x_batched, y_batched):
  # Define the squared loss for a single pair (x,y)
  def squared_error(x, y):
    pred = model.apply(params, x)
    return jnp.inner(y-pred, y-pred) / 2.0
  # Vectorize the previous to compute the average of the loss on all samples.
  return jnp.mean(jax.vmap(squared_error)(x_batched,y_batched), axis=0)

And finally perform the gradient descent.

In [18]:
learning_rate = 0.3  # Gradient step size.
print('Loss for "true" W,b: ', mse(true_params, x_samples, y_samples))
loss_grad_fn = jax.value_and_grad(mse)

@jax.jit
def update_params(params, learning_rate, grads):
  params = jax.tree_util.tree_map(
      lambda p, g: p - learning_rate * g, params, grads) # here we propagate the update to the leaves and save in  *new* params 
  return params

for i in range(101):
  # Perform one gradient update.
  loss_val, grads = loss_grad_fn(params, x_samples, y_samples)
  params = update_params(params, learning_rate, grads)
  if i % 10 == 0:
    print(f'Loss step {i}: ', loss_val)

Loss for "true" W,b:  0.023639796
Loss step 0:  35.343876
Loss step 10:  0.5143469
Loss step 20:  0.113841645
Loss step 30:  0.039326724
Loss step 40:  0.019916208
Loss step 50:  0.014209128
Loss step 60:  0.012425638
Loss step 70:  0.011850387
Loss step 80:  0.0116617745
Loss step 90:  0.011599407
Loss step 100:  0.011578708


### Optimizing with Optax

Flax used to use its own `flax.optim` package for optimization, but with
[FLIP #1009](https://github.com/google/flax/blob/main/docs/flip/1009-optimizer-api.md)
this was deprecated in favor of
[Optax](https://github.com/deepmind/optax).

Basic usage of Optax is straightforward:

1.   Choose an optimization method (e.g. `optax.adam`).
2.   Create optimizer state from parameters (for the Adam optimizer, this state will contain the [momentum values](https://optax.readthedocs.io/en/latest/api.html#optax.adam)).
3.   Compute the gradients of your loss with `jax.value_and_grad()`.
4.   At every iteration, call the Optax `update` function to update the internal
     optimizer state and create an update to the parameters. Then add the update
     to the parameters with Optax's `apply_updates` method.

Note that Optax can do a lot more: it's designed for composing simple gradient
transformations into more complex transformations that allows to implement a
wide range of optimizers. There is also support for changing optimizer
hyperparameters over time ("schedules"), applying different updates to different
parts of the parameter tree ("masking") and much more. For details please refer
to the
[official documentation](https://optax.readthedocs.io/en/latest/).

In [19]:
import optax
tx = optax.adam(learning_rate=learning_rate)
opt_state = tx.init(params)

pytree structure similar to model params

In [34]:
opt_state

(ScaleByAdamState(count=DeviceArray(101, dtype=int32), mu=FrozenDict({
     params: {
         bias: DeviceArray([8.4419778e-05, 8.6254418e-05, 8.4112398e-05, 8.5850959e-05,
                      2.7581875e-04], dtype=float32),
         kernel: DeviceArray([[ 3.7785831e-05,  3.9653976e-05,  3.7664751e-05,
                        3.9230170e-05, -7.9206111e-06],
                      [-4.8984188e-05, -4.6369638e-05, -4.9531027e-05,
                       -4.6984744e-05,  2.3492430e-05],
                      [ 1.3077255e-04,  1.3488121e-04,  1.2978450e-04,
                        1.3396893e-04,  9.9794233e-05],
                      [ 4.6204983e-05,  4.4384080e-05,  4.6581747e-05,
                        4.4750392e-05, -1.7373214e-04],
                      [ 3.1420896e-05,  3.2319105e-05,  3.1092823e-05,
                        3.2122458e-05,  5.1404175e-05],
                      [-1.2748332e-04, -1.2330414e-04, -1.2805173e-04,
                       -1.2431902e-04, -3.2425162e-04],
  

In [39]:
opt_state[0].mu['params']['bias'].shape

(5,)

In [40]:
opt_state[0].mu['params']['kernel'].shape

(10, 5)

In [41]:
opt_state[0].nu['params']['kernel'].shape

(10, 5)

In [42]:
opt_state[0].nu['params']['bias'].shape

(5,)

kernels have same dimension as the kernel of the dense layer. They carry the moving average and squared moving average of all parameters after all. https://arxiv.org/pdf/1609.04747.pdf



essentially the whole network definition (not weights) is contained in the mse function

In [28]:

loss_grad_fn = jax.value_and_grad(mse)

In [29]:
for i in range(101):
  loss_val, grads = loss_grad_fn(params, x_samples, y_samples)
  updates, opt_state = tx.update(grads, opt_state)
  params = optax.apply_updates(params, updates)
  if i % 10 == 0:
    print('Loss step {}: '.format(i), loss_val)

Loss step 0:  0.011577619
Loss step 10:  0.26144287
Loss step 20:  0.07675611
Loss step 30:  0.03644108
Loss step 40:  0.022013972
Loss step 50:  0.016178856
Loss step 60:  0.013002962
Loss step 70:  0.012026138
Loss step 80:  0.011764428
Loss step 90:  0.011646037
Loss step 100:  0.011585514


We could print out the Adam parameters at each epoch via opt_state 

### Serializing the result

Now that we're happy with the result of our training, we might want to save the model parameters to load them back later. Flax provides a serialization package to enable you to do that.

In [43]:
from flax import serialization
bytes_output = serialization.to_bytes(params)
dict_output = serialization.to_state_dict(params)
print('Dict output')
print(dict_output)
print('Bytes output')
print(bytes_output)

Dict output
{'params': {'bias': DeviceArray([-1.4555762 , -2.0277984 ,  2.0790977 ,  1.2186147 ,
             -0.99809736], dtype=float32), 'kernel': DeviceArray([[ 1.009881  ,  0.18934382,  0.04455106, -0.928022  ,
               0.3478402 ],
             [ 1.7298452 ,  0.98793656,  1.1640462 ,  1.1006075 ,
              -0.10653938],
             [-1.2029457 ,  0.28635252,  1.4155988 ,  0.11870988,
              -1.3141484 ],
             [-1.1941482 , -0.18958496,  0.03413888,  1.3169429 ,
               0.08060338],
             [ 0.13852432,  1.3713046 , -1.3187189 ,  0.5315268 ,
              -2.2404993 ],
             [ 0.5629398 ,  0.81223106,  0.3175198 ,  0.5345507 ,
               0.9050035 ],
             [-0.37926078,  1.7410387 ,  1.0790285 , -0.50398403,
               0.92830586],
             [ 0.97064984, -1.3153398 ,  0.3368159 ,  0.809935  ,
              -1.2018455 ],
             [ 1.0194304 , -0.62024873,  1.0818819 , -1.8389751 ,
              -0.4580506 ],
    

To load the model back, you'll need to use a template of the model parameter structure, like the one you would get from the model initialization. Here, we use the previously generated `params` as a template. Note that this will produce a new variable structure, and not mutate in-place.

*The point of enforcing structure through template is to avoid users issues downstream, so you need to first have the right model that generates the parameters structure.*

In [ ]:
serialization.from_bytes(params, bytes_output)

FrozenDict({
    params: {
        bias: array([-1.4540135, -2.0262308,  2.0806582,  1.2201802, -0.9964547],
              dtype=float32),
        kernel: array([[ 1.0106664 ,  0.19014716,  0.04533899, -0.92722285,  0.34720102],
               [ 1.7320251 ,  0.9901233 ,  1.1662225 ,  1.1027892 , -0.10574618],
               [-1.2009128 ,  0.28837162,  1.4176372 ,  0.12073109, -1.3132601 ],
               [-1.1944956 , -0.18993308,  0.03379077,  1.3165942 ,  0.07996067],
               [ 0.14103189,  1.3737966 , -1.3162128 ,  0.53401774, -2.239638  ],
               [ 0.5643044 ,  0.813604  ,  0.31888172,  0.5359193 ,  0.90352124],
               [-0.37948322,  1.7408353 ,  1.0788013 , -0.5041964 ,  0.9286919 ],
               [ 0.9701384 , -1.3158673 ,  0.33630812,  0.80941117, -1.202457  ],
               [ 1.0198247 , -0.6198277 ,  1.0822718 , -1.8385581 , -0.45790705],
               [-0.64384323,  0.4564892 , -1.1331053 , -0.68556863,  0.17010891]],
              dtype=float32),
  

## Defining your own models

Flax allows you to define your own models, which should be a bit more complicated than a linear regression. In this section, we'll show you how to build simple models. To do so, you'll need to create subclasses of the base `nn.Module` class.

*Keep in mind that we imported* `linen as nn` *and this only works with the new linen API*

### Module basics

The base abstraction for models is the `nn.Module` class, and every type of predefined layers in Flax (like the previous `Dense`) is a subclass of `nn.Module`. Let's take a look and start by defining a simple but custom multi-layer perceptron i.e. a sequence of Dense layers interleaved with calls to a non-linear activation function.

In [48]:
class ExplicitMLP(nn.Module):
  features: Sequence[int]

  def setup(self):
    # we automatically know what to do with lists, dicts of submodules
    self.layers = [nn.Dense(feat) for feat in self.features]
    # for single submodules, we would just write:
    # self.layer1 = nn.Dense(feat1)

  def __call__(self, inputs):
    x = inputs
    for i, lyr in enumerate(self.layers):
      x = lyr(x)
      if i != len(self.layers) - 1:
        x = nn.relu(x)
    return x

key1, key2 = random.split(random.PRNGKey(0), 2)
x = random.uniform(key1, (4,4))

model = ExplicitMLP(features=[3,4,5])
params = model.init(key2, x)
y = model.apply(params, x)

print('initialized parameter shapes:\n', jax.tree_util.tree_map(jnp.shape, unfreeze(params)))
print('output:\n', y)

initialized parameter shapes:
 {'params': {'layers_0': {'bias': (3,), 'kernel': (4, 3)}, 'layers_1': {'bias': (4,), 'kernel': (3, 4)}, 'layers_2': {'bias': (5,), 'kernel': (4, 5)}}}
output:
 [[ 0.          0.          0.          0.          0.        ]
 [ 0.00723788 -0.00810345 -0.02550932  0.02151709 -0.01261238]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]


In [49]:
jax.tree_util.tree_map(jnp.shape, unfreeze(params))

{'params': {'layers_0': {'bias': (3,), 'kernel': (4, 3)},
  'layers_1': {'bias': (4,), 'kernel': (3, 4)},
  'layers_2': {'bias': (5,), 'kernel': (4, 5)}}}

In [54]:
y.reshape(-1, 1).shape[0]

20

In [55]:
jnp.reshape

<function jax._src.numpy.lax_numpy.reshape(a: Union[jax._src.basearray.Array, numpy.ndarray, numpy.bool_, numpy.number, bool, int, float, complex], newshape: Union[int, Any, Sequence[Union[int, Any]]], order: str = 'C') -> jax._src.basearray.Array>

In [56]:
jax.tree_util.tree_map(lambda x: jnp.reshape(x, (-1, 1))[0], unfreeze(params))

{'params': {'layers_0': {'bias': DeviceArray([0.], dtype=float32),
   'kernel': DeviceArray([0.15538356], dtype=float32)},
  'layers_1': {'bias': DeviceArray([0.], dtype=float32),
   'kernel': DeviceArray([-1.035799], dtype=float32)},
  'layers_2': {'bias': DeviceArray([0.], dtype=float32),
   'kernel': DeviceArray([-0.6535059], dtype=float32)}}}

In [57]:
jax.tree_util.tree_map(lambda x: jnp.reshape(x, (-1, 1))[0], params)

FrozenDict({
    params: {
        layers_0: {
            bias: DeviceArray([0.], dtype=float32),
            kernel: DeviceArray([0.15538356], dtype=float32),
        },
        layers_1: {
            bias: DeviceArray([0.], dtype=float32),
            kernel: DeviceArray([-1.035799], dtype=float32),
        },
        layers_2: {
            bias: DeviceArray([0.], dtype=float32),
            kernel: DeviceArray([-0.6535059], dtype=float32),
        },
    },
})

As we can see, a `nn.Module` subclass is made of:

*   A collection of data fields (`nn.Module` are Python dataclasses) - here we only have the `features` field of type `Sequence[int]`.
*   A `setup()` method that is being called at the end of the `__postinit__` where you can register submodules, variables, parameters you will need in your model.
*   A `__call__` function that returns the output of the model from a given input.
*   The model structure defines a pytree of parameters following the same tree structure as the model: the params tree contains one `layers_n` sub dict per layer, and each of those contain the parameters of the associated Dense layer. The layout is very explicit.

*Note: lists are mostly managed as you would expect (WIP), there are corner cases you should be aware of as pointed out* [here](https://github.com/google/flax/issues/524)

Since the module structure and its parameters are not tied to each other, you can't directly call `model(x)` on a given input as it will return an error. The `__call__` function is being wrapped up in the `apply` one, which is the one to call on an input:

In [58]:
try:
    y = model(x) # Returns an error
except AttributeError as e:
    print(e)

"ExplicitMLP" object has no attribute "layers". If "layers" is defined in '.setup()', remember these fields are only accessible from inside 'init' or 'apply'.


Since here we have a very simple model, we could have used an alternative (but equivalent) way of declaring the submodules inline in the `__call__` using the `@nn.compact` annotation like so:

In [59]:
class SimpleMLP(nn.Module):
  features: Sequence[int]

  @nn.compact
  def __call__(self, inputs):
    x = inputs
    for i, feat in enumerate(self.features):
      x = nn.Dense(feat, name=f'layers_{i}')(x)
      if i != len(self.features) - 1:
        x = nn.relu(x)
      # providing a name is optional though!
      # the default autonames would be "Dense_0", "Dense_1", ...
    return x

key1, key2 = random.split(random.PRNGKey(0), 2)
x = random.uniform(key1, (4,4))

model = SimpleMLP(features=[3,4,5])
params = model.init(key2, x)
y = model.apply(params, x)

print('initialized parameter shapes:\n', jax.tree_util.tree_map(jnp.shape, unfreeze(params)))
print('output:\n', y)

initialized parameter shapes:
 {'params': {'layers_0': {'bias': (3,), 'kernel': (4, 3)}, 'layers_1': {'bias': (4,), 'kernel': (3, 4)}, 'layers_2': {'bias': (5,), 'kernel': (4, 5)}}}
output:
 [[ 0.          0.          0.          0.          0.        ]
 [ 0.00723788 -0.00810345 -0.02550932  0.02151709 -0.01261238]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]


There are, however, a few differences you should be aware of between the two declaration modes:

*   In `setup`, you are able to name some sublayers and keep them around for further use (e.g. encoder/decoder methods in autoencoders).
*   If you want to have multiple methods, then you **need** to declare the module using `setup`, as the `@nn.compact` annotation only allows one method to be annotated.
*   The last initialization will be handled differently. See these notes for more details (TODO: add notes link).

### Module parameters

In the previous MLP example, we relied only on predefined layers and operators (`Dense`, `relu`). Let's imagine that you didn't have a Dense layer provided by Flax and you wanted to write it on your own. Here is what it would look like using the `@nn.compact` way to declare a new modules:

In [60]:
class SimpleDense(nn.Module):
  features: int
  kernel_init: Callable = nn.initializers.lecun_normal()
  bias_init: Callable = nn.initializers.zeros

  @nn.compact
  def __call__(self, inputs):
    kernel = self.param('kernel',
                        self.kernel_init, # Initialization function
                        (inputs.shape[-1], self.features))  # shape info.
    y = lax.dot_general(inputs, kernel,
                        (((inputs.ndim - 1,), (0,)), ((), ())),) # TODO Why not jnp.dot?
    bias = self.param('bias', self.bias_init, (self.features,))
    y = y + bias
    return y

key1, key2 = random.split(random.PRNGKey(0), 2)
x = random.uniform(key1, (4,4))

model = SimpleDense(features=3)
params = model.init(key2, x)
y = model.apply(params, x)

print('initialized parameters:\n', params)
print('output:\n', y)

initialized parameters:
 FrozenDict({
    params: {
        kernel: DeviceArray([[ 0.61506   , -0.22728713,  0.60547006],
                     [-0.2961799 ,  1.1232013 , -0.879759  ],
                     [-0.35162622,  0.38064912,  0.68932474],
                     [-0.1151355 ,  0.04567899, -1.091212  ]], dtype=float32),
        bias: DeviceArray([0., 0., 0.], dtype=float32),
    },
})
output:
 [[-0.02996201  1.102088   -0.6660265 ]
 [-0.31092793  0.6323942  -0.53678817]
 [ 0.0142401   0.94247174 -0.63561475]
 [ 0.36818963  0.3586519  -0.00459225]]


Here, we see how to both declare and assign a parameter to the model using the `self.param` method. It takes as input `(name, init_fn, *init_args)` :

*   `name` is simply the name of the parameter that will end up in the parameter structure.
*   `init_fn` is a function with input `(PRNGKey, *init_args)` returning an Array, with `init_args` being the arguments needed to call the initialisation function.
*   `init_args` are the arguments to provide to the initialization function.

Such params can also be declared in the `setup` method; it won't be able to use shape inference because Flax is using lazy initialization at the first call site.

### Variables and collections of variables

As we've seen so far, working with models means working with:

*   A subclass of `nn.Module`;
*   A pytree of parameters for the model (typically from `model.init()`);

However this is not enough to cover everything that we would need for machine learning, especially neural networks. In some cases, you might want your neural network to keep track of some internal state while it runs (e.g. batch normalization layers). There is a way to declare variables beyond the parameters of the model with the `variable` method.

For demonstration purposes, we'll implement a simplified but similar mechanism to batch normalization: we'll store running averages and subtract those to the input at training time. For proper batchnorm, you should use (and look at) the implementation [here](https://github.com/google/flax/blob/main/flax/linen/normalization.py).

In [ ]:
class BiasAdderWithRunningMean(nn.Module):
  decay: float = 0.99

  @nn.compact
  def __call__(self, x):
    # easy pattern to detect if we're initializing via empty variable tree
    is_initialized = self.has_variable('batch_stats', 'mean')
    ra_mean = self.variable('batch_stats', 'mean',
                            lambda s: jnp.zeros(s),
                            x.shape[1:])
    mean = ra_mean.value # This will either get the value or trigger init
    bias = self.param('bias', lambda rng, shape: jnp.zeros(shape), x.shape[1:])
    if is_initialized:
      ra_mean.value = self.decay * ra_mean.value + (1.0 - self.decay) * jnp.mean(x, axis=0, keepdims=True)

    return x - ra_mean.value + bias


key1, key2 = random.split(random.PRNGKey(0), 2)
x = jnp.ones((10,5))
model = BiasAdderWithRunningMean()
variables = model.init(key1, x)
print('initialized variables:\n', variables)
y, updated_state = model.apply(variables, x, mutable=['batch_stats'])
print('updated state:\n', updated_state)

initialized variables:
 FrozenDict({
    batch_stats: {
        mean: DeviceArray([0., 0., 0., 0., 0.], dtype=float32),
    },
    params: {
        bias: DeviceArray([0., 0., 0., 0., 0.], dtype=float32),
    },
})
updated state:
 FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.01, 0.01, 0.01, 0.01, 0.01]], dtype=float32),
    },
})


Here, `updated_state` returns only the state variables that are being mutated by the model while applying it on data. To update the variables and get the new parameters of the model, we can use the following pattern:

In [ ]:
for val in [1.0, 2.0, 3.0]:
  x = val * jnp.ones((10,5))
  y, updated_state = model.apply(variables, x, mutable=['batch_stats'])
  old_state, params = variables.pop('params')
  variables = freeze({'params': params, **updated_state})
  print('updated state:\n', updated_state) # Shows only the mutable part

updated state:
 FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.01, 0.01, 0.01, 0.01, 0.01]], dtype=float32),
    },
})
updated state:
 FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.0299, 0.0299, 0.0299, 0.0299, 0.0299]], dtype=float32),
    },
})
updated state:
 FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.059601, 0.059601, 0.059601, 0.059601, 0.059601]], dtype=float32),
    },
})


From this simplified example, you should be able to derive a full BatchNorm implementation, or any layer involving a state. To finish, let's add an optimizer to see how to play with both parameters updated by an optimizer and state variables.

*This example isn't doing anything and is only for demonstration purposes.*

In [ ]:
from functools import partial

@partial(jax.jit, static_argnums=(0, 1))
def update_step(tx, apply_fn, x, opt_state, params, state):

  def loss(params):
    y, updated_state = apply_fn({'params': params, **state},
                                x, mutable=list(state.keys()))
    l = ((x - y) ** 2).sum()
    return l, updated_state

  (l, state), grads = jax.value_and_grad(loss, has_aux=True)(params)
  updates, opt_state = tx.update(grads, opt_state)
  params = optax.apply_updates(params, updates)
  return opt_state, params, state

x = jnp.ones((10,5))
variables = model.init(random.PRNGKey(0), x)
state, params = variables.pop('params')
del variables
tx = optax.sgd(learning_rate=0.02)
opt_state = tx.init(params)

for _ in range(3):
  opt_state, params, state = update_step(tx, model.apply, x, opt_state, params, state)
  print('Updated state: ', state)

Updated state:  FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.01, 0.01, 0.01, 0.01, 0.01]], dtype=float32),
    },
})
Updated state:  FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.0199, 0.0199, 0.0199, 0.0199, 0.0199]], dtype=float32),
    },
})
Updated state:  FrozenDict({
    batch_stats: {
        mean: DeviceArray([[0.029701, 0.029701, 0.029701, 0.029701, 0.029701]], dtype=float32),
    },
})


Note that the above function has a quite verbose signature and it would not actually
work with `jax.jit()` because the function arguments are not "valid JAX types".

Flax provides a handy wrapper - `TrainState` - that simplifies the above code. Check out [`flax.training.train_state.TrainState`](https://flax.readthedocs.io/en/latest/api_reference/flax.training.html#flax.training.train_state.TrainState) to learn more.

### Exporting to Tensorflow's SavedModel with jax2tf

JAX released an experimental converter called [jax2tf](https://github.com/google/jax/tree/main/jax/experimental/jax2tf), which allows converting trained Flax models into Tensorflow's SavedModel format (so it can be used for [TF Hub](https://www.tensorflow.org/hub), [TF.lite](https://www.tensorflow.org/lite), [TF.js](https://www.tensorflow.org/js), or other downstream applications). The repository contains more documentation and has various examples for Flax.